# Import the libraries and set the random seed.

In [1]:
# Import the libraries and set the random seed

from torch import nn
import torch
torch.manual_seed(1)

<!--Empty Space for separating topics-->

<h2 id="Prediction">Prediction</h2>

Set weight and bias.

In [2]:
# Set the weight and bias

w = torch.tensor([[2.0], [3.0]], requires_grad=True)
b = torch.tensor([[1.0]], requires_grad=True)

Define the parameters. <code>torch.mm</code> uses matrix multiplication instead of scaler multiplication.

In [3]:
# Define Prediction Function

def forward(x):
    yhat = torch.mm(x, w) + b
    return yhat

If we input a <i>1x2</i> tensor, because we have a <i>2x1</i> tensor as <code>w</code>, we will get a <i>1x1</i> tensor: 

In [4]:
# Calculate yhat

x = torch.tensor([[1.0, 2.0]])
yhat = forward(x)
print("The result: ", yhat)

The result:  tensor([[9.]], grad_fn=<AddBackward0>)


# Each row of the following tensor represents a sample:

In [5]:
# Sample tensor X

X = torch.tensor([[1.0, 1.0], [1.0, 2.0], [1.0, 3.0]])

In [6]:
# Make the prediction of X 

yhat = forward(X)
print("The result: ", yhat)

The result:  tensor([[ 6.],
        [ 9.],
        [12.]], grad_fn=<AddBackward0>)


<!--Empty Space for separating topics-->

<h2 id="Linear">Class Linear</h2>

We can use the linear class to make a prediction. You'll also use the linear class to build more complex models.

Let us create a model.

In [7]:
# Make a linear regression model using build-in function

model = nn.Linear(2, 1)

Make a prediction with the first sample:

In [8]:
# Make a prediction of x

yhat = model(x)
print("The result: ", yhat)

The result:  tensor([[-0.3969]], grad_fn=<AddmmBackward>)


Predict with multiple samples <code>X</code>: 

In [9]:
# Make a prediction of X

yhat = model(X)
print("The result: ", yhat)

The result:  tensor([[-0.0848],
        [-0.3969],
        [-0.7090]], grad_fn=<AddmmBackward>)
